# Initial Allignment Assistant

## Run this bit of code first

In [1]:
#Importing sys so we can importy Optics.py
import sys
sys.path.insert(0, '~/Initial-Allignment-Assistant-Tool')
#Importing all the definitions form Optics.py
from Optics import *

import Parameters

#Imports numpy
import numpy as np
#Imports improtant functions for juptyer display
from IPython.core.display import display, HTML, Markdown, clear_output
#Improts matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
#Improts widgets for buttons and text boxes for jupyter display
import ipywidgets as widgets
#Imports parser for ini files
import configparser
#Import os to see files and directories
import os

/home/andre.guimaraes/.conda/envs/IAAT-Env/lib/python3.7/site-packages/traitlets/traitlets.py:567: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


In [2]:
def resetPlot(ax):
    #clears everything (that was defined on ax, which is everything)
    ax.cla()
    #Sets aspect ratio to 1
    ax.set_aspect(1)
    #Drags left and bottom axis to the center.
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')
    # Eliminate upper and right axes.
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')

#Function to draw a circle on the plot
def drawCircle(ax, center_xy = [0, 0], radius = 1, color = 'black', alpha = 1, linewidth = 1):
    ax.add_artist(plt.Circle(center_xy, radius, color = color, fill = False, alpha = alpha, linewidth = linewidth))

#Function to draw a disk on the plot
def drawDisk(ax, center_xy = [0, 0], radius = 1, color = 'black', alpha = 1):
    ax.add_artist(plt.Circle(center_xy, radius, color = color, fill = True, alpha = alpha))

#Function to draw a line on the plot
def drawLine(ax, minmax_x = [0, 1], minmax_y = [0, 1], color = 'black', alpha = 1):
    ax.plot(minmax_x, minmax_y, color = color, alpha = alpha)
    
#Function to calculate all that is needed for the plot, returns a dictionary array.
def calculatePlotParameters(beam0, elements0, elementView, elementControl, extraYaws, extraPitches):
    #Makes a copy of the beam
    beam = beam0.copy()
    beamDefault = beam0.copy()
    #Makes a copy of all the elements
    elements = [element.copy() for element in elements0]
    elementsDefault = [element.copy() for element in elements0]

    #Shifts the pitch and yaw of all the elements accordingly
    for i in range(len(elements)):
        elements[i].pitch = elements[i].pitch + extraPitches[i]
        elements[i].yaw = elements[i].yaw + extraYaws[i]
    #Finds the ID os the element that is being controlled (not useful yet, but no harm in it).
    for iCtrl in range(len(elements)):
        if elements[iCtrl].ID == elementControl:
            break
    #Finds the ID os the element that is being viewed.
    for iView in range(len(elements)):
        if elements[iView].ID == elementView:
            break
            
    #Calculates the states of the beam at each element.
    states = beam.calculateStates(elements)
    statesDefault = beam.calculateStates(elementsDefault)
    #Calculates the flags for non-intersection and clipping
    flags = beam.calculateFlags(elements)
    
    #Returns the dictionary with all necessary information for the plot.
    return {
            "Title": "View: " + str(elementView) + " | Control: " + str(elementControl),
            "Range": np.array([-1, 1])*elements[iView].diameter*1.1/2.0,
            "BeamCenter": rotatePitchYaw(states[elementView].position - elements[iView].positionOfCM, -elements[iView].pitch, -elements[iView].yaw)[1:],
            "BeamRadius": states[elementView].width,
            "BeamCenterDefault": rotatePitchYaw(statesDefault[elementView].position - elementsDefault[iView].positionOfCM, -elementsDefault[iView].pitch, -elementsDefault[iView].yaw)[1:],
            "BeamRadiusDefault": statesDefault[elementView].width,
            "Flags": flags            
           }

#Function to draw the plot based on the parameters given.
def drawState(ax, plotParameters, optionsDict = {'showDefaultBeam': False, 'showDisplacement': False, 'flagFontSize': 12, 'plotFontSize': 12, 'axesTicks': 7, 'numberOfCircles': 10, 'numberOfLines': 16, 'beamColor': 'Red'}):
    showDefaultBeam = optionsDict['showDefaultBeam']
    showDisplacement = optionsDict['showDisplacement']
    plotFontSize = optionsDict['plotFontSize']
    flagFontSize = optionsDict['flagFontSize']
    numberOfTicks = optionsDict['axesTicks']
    #Restarts the plot completely
    resetPlot(ax)
    #Setting some parameters
    ax.set_title(plotParameters['Title'], fontsize = int(plotFontSize*1.5))
    ax.set_xlim(plotParameters['Range'])
    ax.set_ylim(plotParameters['Range'])
    ax.xaxis.set_ticks(np.arange(plotParameters['Range'][0]/1.1, plotParameters['Range'][1]/1.1+1, (plotParameters['Range'][1] - plotParameters['Range'][0])/(1.1*(numberOfTicks-1))))
    ax.tick_params(axis='x', which = 'major', rotation = 45)
    ax.yaxis.set_ticks(np.arange(plotParameters['Range'][0]/1.1, plotParameters['Range'][1]/1.1+1, (plotParameters['Range'][1] - plotParameters['Range'][0])/(1.1*(numberOfTicks-1))))
    ax.tick_params(axis="x", labelsize = plotFontSize)
    ax.tick_params(axis="y", labelsize = plotFontSize)
    #Drawing constant radius lines
    for i in range(optionsDict['numberOfCircles']):
        drawCircle(ax, [0, 0], plotParameters['Range'][0]/1.1/optionsDict['numberOfCircles']*(i), alpha = .3)
    #Draws edge of element
    drawCircle(ax, [0, 0], plotParameters['Range'][0]/1.1/optionsDict['numberOfCircles']*(optionsDict['numberOfCircles']), linewidth = 2)
    
    #Drawing constant angle lines
    for i in range(optionsDict['numberOfLines']):
        drawLine(ax, plotParameters['Range'][0]/1.1*np.array([0,np.cos(2.0*np.pi/optionsDict['numberOfLines']*i)]),plotParameters['Range'][0]/1.1*np.array([0,np.sin(2.0*np.pi/optionsDict['numberOfLines']*i)]), alpha = .3)
    
    #Code to execute of the "Show Default Beam" checkbox is clicked
    if showDefaultBeam:
        #Draws default beam
        drawDisk(ax, center_xy = plotParameters['BeamCenterDefault'], radius = plotParameters['BeamRadiusDefault'], color = 'black', alpha = .25)
        drawDisk(ax, center_xy = plotParameters['BeamCenterDefault'], radius = plotParameters['BeamRadiusDefault']/3.0, color = 'black', alpha = .375)
        drawDisk(ax, center_xy = plotParameters['BeamCenterDefault'], radius = plotParameters['BeamRadiusDefault']/50.0, color = 'black', alpha = .5)
    if showDisplacement:
        #Draws line form the cenrter of the default beam to the center of the new beam
        #drawLine(ax, [plotParameters['BeamCenterDefault'][0],plotParameters['BeamCenter'][0]],[plotParameters['BeamCenterDefault'][1],plotParameters['BeamCenter'][1]], color = 'blue', alpha = .5)
        drawLine(ax, [0, plotParameters['BeamCenter'][0]],[0 ,plotParameters['BeamCenter'][1]], color = 'blue', alpha = .5)
        #ax.text((plotParameters['BeamCenter']-plotParameters['BeamCenterDefault'])[0]/2.0, (plotParameters['BeamCenter']-plotParameters['BeamCenterDefault'])[1]/2.0, "\u0394S = " + str(norm(plotParameters['BeamCenter']-plotParameters['BeamCenterDefault'])), color = 'blue', alpha = 1, fontsize = plotFontSize)
        ax.text(plotParameters['BeamCenter'][0], plotParameters['BeamCenter'][1], "\u0394S = " + str(norm(plotParameters['BeamCenter'])), color = 'blue', alpha = 1, fontsize = plotFontSize)
    #Draws beam spot at twice width
    #drawDisk(ax, center_xy = plotParameters['BeamCenter'], radius = 2*plotParameters['BeamRadius'], color = optionsDict['beamColor'], alpha = .3)
    #Draws beam spot at full width
    drawDisk(ax, center_xy = plotParameters['BeamCenter'], radius = plotParameters['BeamRadius'], color = optionsDict['beamColor'], alpha = .5)
    #Draws beam spot at 1/2 width
    drawDisk(ax, center_xy = plotParameters['BeamCenter'], radius = plotParameters['BeamRadius']/2.0, color = optionsDict['beamColor'], alpha = .75)
    #Draws center of beam spot (1/50 width)
    drawDisk(ax, center_xy = plotParameters['BeamCenter'], radius = plotParameters['BeamRadius']/50.0, color = 'black', alpha = 1)

    #Defines the text of all the flags.
    flagText = ""
    if len(plotParameters['Flags']) >= 1:
        for flag in plotParameters['Flags']:
            flagText = flagText + flag + '\n'
    #Prints the flags
    ax.text(-plotParameters['Range'][0], -plotParameters['Range'][0], flagText, color = 'red', fontsize = flagFontSize)

def drawXYView(ax, plotParameters, optionsDict, ):
    showDefaultBeam = optionsDict['showDefaultBeam']
    showDisplacement = optionsDict['showDisplacement']
    plotFontSize = optionsDict['plotFontSize']
    flagFontSize = optionsDict['flagFontSize']
    numberOfTicks = optionsDict['axesTicks']
    x_i = np.min([element.positionOfCM[0] - element.diameter for element in Parameters.elements])
    x_i = np.min([x_i, Parameters.beam.position[0]])
    x_f = np.max([element.positionOfCM[0] + element.diameter for element in Parameters.elements])
    x_f = np.max([x_f, Parameters.beam.position[0]])
    

    x_i2 = x_i - .25*(x_f-x_i)
    x_f2 = x_f + .25*(x_f-x_i)
    
    y_i = np.min([element.positionOfCM[1] - element.diameter for element in Parameters.elements])
    y_i = np.min([y_i, Parameters.beam.position[1]])
    y_f = np.max([element.positionOfCM[1] + element.diameter for element in Parameters.elements])
    y_f = np.max([y_f, Parameters.beam.position[1]])
    
    y_i2 = y_i - .25*(y_f-y_i)
    y_f2 = y_f + .25*(y_f-y_i)
    
    range_x = np.array([x_i2, x_f2])
    range_y = np.array([y_i2, y_f2])
    
    ticks_x = np.arange(range_x[0], range_x[1]+1, (range_x[1] - range_x[0])/(numberOfTicks-1))
    ticks_y = np.arange(range_y[0], range_y[1]+1, (range_y[1] - range_y[0])/(numberOfTicks-1))
    
    resetPlot(ax)
    #Setting some parameters
    ax.set_aspect('auto')
    ax.set_title(plotParameters['Title'] + ' - x-y Plane', fontsize = int(plotFontSize*1.5))
    ax.set_xlim(range_x)
    ax.set_ylim(range_y)
    ax.xaxis.set_ticks(ticks_x)
    ax.tick_params(axis='x', which = 'major', rotation = 45)
    ax.yaxis.set_ticks(ticks_y)
    ax.tick_params(axis="x", labelsize = plotFontSize)
    ax.tick_params(axis="y", labelsize = plotFontSize)

    for i in range(len(ticks_x)):
        drawLine(ax, [ticks_x[i], ticks_x[i]], [ticks_y[0],ticks_y[-1]], alpha = .3)
    for i in range(len(ticks_y)):
        drawLine(ax, [ticks_x[0], ticks_x[-1]], [ticks_y[i],ticks_y[i]], alpha = .3)
    
    for element in Parameters.elements:
        if isinstance(element, Mirror):
            thetaVertex = np.pi - angleBetweenVectors(np.array([1,0,0]), np.append((element.center() - element.vertex())[:2],0))
            if (element.vertex() - element.center())[1] < 0:
                thetaVertex = -thetaVertex
            theta1 = (thetaVertex - element.diameter/(2.0*element.radiusOfCurvature))*180/np.pi
            theta2 = (thetaVertex + element.diameter/(2.0*element.radiusOfCurvature))*180/np.pi
            ax.add_patch(Arc(element.center()[:2], 2.0*element.radiusOfCurvature, 2.0*element.radiusOfCurvature, theta1 = theta1, theta2 = theta2))
        if isinstance(element, FlatMirror) or isinstance(element, WedgePolarizer):
            point0 = element.vertex1()[:2]
            point1 = point0 + element.diameter/2.0*np.array([np.sin(element.yaw), np.cos(element.yaw)])
            point2 = point0 - element.diameter/2.0*np.array([np.sin(element.yaw), np.cos(element.yaw)])
            drawLine(ax, [point1[0],point2[0]], [point1[1],point2[1]])
        elif isinstance(element, Lens):
            thetaVertex = np.pi - angleBetweenVectors(np.array([1,0,0]), np.append((element.vertex1() - element.center1())[:2],0))
            if (element.vertex1() - element.center1())[1] < 0:
                thetaVertex = -thetaVertex
            theta1 = (thetaVertex - element.diameter/(2.0*element.radiusOfCurvature))*180/np.pi
            theta2 = (thetaVertex + element.diameter/(2.0*element.radiusOfCurvature))*180/np.pi
            #ax.add_patch(Arc(element.center1()[:2], 2.0*element.radiusOfCurvature, 2.0*element.radiusOfCurvature, theta1 = np.pi + theta1, theta2 = np.pi + theta2))
            ax.add_patch(Arc(element.center2()[:2], 2.0*element.radiusOfCurvature, 2.0*element.radiusOfCurvature, theta1 = theta1, theta2 = theta2))
    
    beam0 = Parameters.beam.copy()
    elements0 = [Parameters.elements[i].copy() for i in range(len(Parameters.elements))]
    
    for i in range(len(elements0)):
        elements0[i].pitch = elements0[i].pitch + Parameters.extraPitches[i]
        elements0[i].yaw = elements0[i].yaw + Parameters.extraYaws[i]
    states = beam0.calculateStates(elements0)
    
    drawLine(ax, [states['Source'].position[0], states[elements0[0].ID].position[0]], [states['Source'].position[1], states[elements0[0].ID].position[1]], color = 'red')
    
    for i in range(len(elements0) - 1):
        drawLine(ax, [states[elements0[i].ID].position[0], states[elements0[i+1].ID].position[0]], [states[elements0[i].ID].position[1], states[elements0[i + 1].ID].position[1]], color = 'red')
        
#Calculates the multiplier based on the units selected
def calculateMultiplier(dropdownValue):
    if dropdownValue == 'Radians':
        return 1.0
    elif dropdownValue == 'Milli Radians':
        return 1.0e-3
    elif dropdownValue == 'Micro Radians':
        return 1.0e-6
    elif dropdownValue == 'Nano Radians':
        return 1.0e-9
    else:
        return 0



In [3]:
def showPlotClick(b):
    #Tells what the output is
    with output:
        #Claculates the multiplier of the yaw based in the units selected.
        yawMultiplier = calculateMultiplier(Parameters.yawUnitsDropdown.value)
        #Changes the yaw of the mirror being controlled
        Parameters.extraYaws[Parameters.controlIndex] = float(Parameters.yawTextBox.value)*yawMultiplier
        #Claculates the multiplier of the pitch based in the units selected.
        pitchMultiplier = calculateMultiplier(Parameters.pitchUnitsDropdown.value)
        #Changes the pitch of the mirror being controlled
        Parameters.extraPitches[Parameters.controlIndex] = float(Parameters.pitchTextBox.value)*pitchMultiplier

        #Calculates all the parameters needed for the plot with the new pitches and yaws
        params = calculatePlotParameters(beam0 = Parameters.beam, elements0 = Parameters.elements, elementView = Parameters.elementView, elementControl = Parameters.elementControl, extraYaws = Parameters.extraYaws, extraPitches = Parameters.extraPitches)
        #Initialized the plot again
        fig = plt.figure(figsize=(15, 10), dpi= 80, facecolor='w', edgecolor='k')
        ax = plt.gca()

        #Clears all the output
        clear_output()
        #Defines dictionary with all the formatting options
        optionsDict =  {
            'showDefaultBeam': int(Parameters.showDefaultBeamCheckbox.value),
            'showDisplacement': int(Parameters.showDisplacementCheckbox.value),
            'plotFontSize': int(Parameters.plotFontSizeTextBox.value),
            'flagFontSize': int(Parameters.flagFontSizeTextBox.value),
            'axesTicks': int(Parameters.axesTicksTextBox.value),
            'numberOfCircles': int(Parameters.numberOfCirclesTextBox.value),
            'numberOfLines': int(Parameters.numberOfLinesTextBox.value),
            'beamColor': Parameters.beamColorDropdown.value
        }
        
        
        drawState(ax, params, optionsDict = optionsDict)
        if Parameters.viewXYPlaneCheckbox.value:
            drawXYView(ax, params, optionsDict)
        plt.show()

def setElementClick(b):
    with output:
        Parameters.elementView = Parameters.elementViewDropdown.value
        
        Parameters.elementControl = Parameters.elementControlDropdown.value
        
        Parameters.viewIndex = np.argwhere(Parameters.elementsIDs == Parameters.elementView)[0][0]
        
        Parameters.controlIndex = np.argwhere(Parameters.elementsIDs == Parameters.elementControl)[0][0]
        
        #Sets the textBox for the yaw to be the yaw the new controlled element has
        Parameters.yawTextBox.value = str(Parameters.extraYaws[Parameters.controlIndex])
        #Sets the textBox for the pitch to be the yaw the new controlled element has
        Parameters.pitchTextBox.value = str(Parameters.extraPitches[Parameters.controlIndex])
        
        #Resets unit boxes to Radians
        Parameters.yawUnitsDropdown.value = 'Radians'
        Parameters.pitchUnitsDropdown.value = 'Radians'
        #showPlotClickFunc()
        
        #showPlotClick(1)
        #Claculates the multiplier of the yaw based in the units selected.
        yawMultiplier = calculateMultiplier(Parameters.yawUnitsDropdown.value)
        #Changes the yaw of the mirror being controlled
        Parameters.extraYaws[Parameters.controlIndex] = float(Parameters.yawTextBox.value)*yawMultiplier
        #Claculates the multiplier of the pitch based in the units selected.
        pitchMultiplier = calculateMultiplier(Parameters.pitchUnitsDropdown.value)
        #Changes the pitch of the mirror being controlled
        Parameters.extraPitches[Parameters.controlIndex] = float(Parameters.pitchTextBox.value)*pitchMultiplier
        
        #Calculates all the parameters needed for the plot with the new pitches and yaws
        params = calculatePlotParameters(beam0 = Parameters.beam, elements0 = Parameters.elements, elementView = Parameters.elementView, elementControl = Parameters.elementControl, extraYaws = Parameters.extraYaws, extraPitches = Parameters.extraPitches)
        #Initialized the plot again
        fig = plt.figure(figsize=(15, 10), dpi= 80, facecolor='w', edgecolor='k')
        ax = plt.gca()

        #Clears all the output
        clear_output()
        #Defines dictionary with all the formatting options
        optionsDict =  {
            'showDefaultBeam': int(Parameters.showDefaultBeamCheckbox.value),
            'showDisplacement': int(Parameters.showDisplacementCheckbox.value),
            'plotFontSize': int(Parameters.plotFontSizeTextBox.value),
            'flagFontSize': int(Parameters.flagFontSizeTextBox.value),
            'axesTicks': int(Parameters.axesTicksTextBox.value),
            'numberOfCircles': int(Parameters.numberOfCirclesTextBox.value),
            'numberOfLines': int(Parameters.numberOfLinesTextBox.value),
            'beamColor': Parameters.beamColorDropdown.value
        }
        #Draws new plot
        drawState(ax, params, optionsDict = optionsDict)
        #Shows plot
        plt.show()
    
def selectSystemClick(b):
    with output:
        
        for i in range(len(Parameters.systems)):
            if Parameters.systems[i] == Parameters.systemSelectionDropdown.value:
                Parameters.systemIndex = i
                break
        
        Parameters.beam = Parameters.beamFromFile('Systems/' + Parameters.systems[Parameters.systemIndex] + '.ini')
        #global elements
        Parameters.elements = Parameters.elementsFromFile('Systems/' + Parameters.systems[Parameters.systemIndex] + '.ini')

        #Makes list of the ID of the optical elements.
        #global elementsIDs
        Parameters.elementsIDs = np.array([element.ID for element in Parameters.elements])
        #Default element to be viewed (last in the series).
        Parameters.elementView = Parameters.elementsIDs[-1]
        #Default element to be controlled (last in the series).
        Parameters.elementControl = Parameters.elementsIDs[0]

        #Finds index of the element being viewed.
        Parameters.viewIndex = np.argwhere(Parameters.elementsIDs == Parameters.elementView)[0][0]
        #Finds index of the element being controlled.
        Parameters.controlIndex = np.argwhere(Parameters.elementsIDs == Parameters.elementControl)[0][0]

        #Initialized the "extra Yaws" to be zero for all elements.
        #global extraYaws
        Parameters.extraYaws = np.array([0.0 for element in Parameters.elements])
        #Initialized the "extra Pitches" to be zero for all elements.
        #global extraPitches
        Parameters.extraPitches = np.array([0.0 for element in Parameters.elements])

        #Sets the textBox for the yaw to be the yaw the new controlled element has
        Parameters.yawTextBox.value = str(Parameters.extraYaws[Parameters.controlIndex])
        #Sets the textBox for the pitch to be the yaw the new controlled element has
        Parameters.pitchTextBox.value = str(Parameters.extraPitches[Parameters.controlIndex])
        
        Parameters.elementViewDropdown.options = Parameters.elementsIDs
        Parameters.elementViewDropdown.value = Parameters.elementView
        
        Parameters.elementControlDropdown.options = Parameters.elementsIDs
        Parameters.elementControlDropdown.value = Parameters.elementControl        
        
        #Resets unit boxes to Radians
        Parameters.yawUnitsDropdown.value = 'Radians'
        Parameters.pitchUnitsDropdown.value = 'Radians'
        #showPlotClickFunc()
        
        #showPlotClick(1)
        #Claculates the multiplier of the yaw based in the units selected.
        yawMultiplier = calculateMultiplier(Parameters.yawUnitsDropdown.value)
        #Changes the yaw of the mirror being controlled
        Parameters.extraYaws[Parameters.controlIndex] = float(Parameters.yawTextBox.value)*yawMultiplier
        #Claculates the multiplier of the pitch based in the units selected.
        pitchMultiplier = calculateMultiplier(Parameters.pitchUnitsDropdown.value)
        #Changes the pitch of the mirror being controlled
        Parameters.extraPitches[Parameters.controlIndex] = float(Parameters.pitchTextBox.value)*pitchMultiplier
        
        #Calculates all the parameters needed for the plot with the new pitches and yaws
        params = calculatePlotParameters(beam0 = Parameters.beam, elements0 = Parameters.elements, elementView = Parameters.elementView, elementControl = Parameters.elementControl, extraYaws = Parameters.extraYaws, extraPitches = Parameters.extraPitches)
        #Initialized the plot again
        fig = plt.figure(figsize=(15, 10), dpi= 80, facecolor='w', edgecolor='k')
        ax = plt.gca()

        #Clears all the output
        clear_output()
        #Defines dictionary with all the formatting options
        optionsDict =  {
            'showDefaultBeam': int(Parameters.showDefaultBeamCheckbox.value),
            'showDisplacement': int(Parameters.showDisplacementCheckbox.value),
            'plotFontSize': int(Parameters.plotFontSizeTextBox.value),
            'flagFontSize': int(Parameters.flagFontSizeTextBox.value),
            'axesTicks': int(Parameters.axesTicksTextBox.value),
            'numberOfCircles': int(Parameters.numberOfCirclesTextBox.value),
            'numberOfLines': int(Parameters.numberOfLinesTextBox.value),
            'beamColor': Parameters.beamColorDropdown.value
        }
        #Draws new plot
        drawState(ax, params, optionsDict = optionsDict)
        
        #Shows plot
        plt.show()



## The Actual Assistant

In [4]:
#Initialized plot.
global fig
fig = plt.figure(figsize=(15, 10), dpi= 80, facecolor='w', edgecolor='k');
global ax
ax = plt.gca();

Parameters.resetWidgets()

#Needed just because.
#global output
output = widgets.Output()

params = calculatePlotParameters(beam0 = Parameters.beam, elements0 = Parameters.elements, elementView = Parameters.elementView, elementControl = Parameters.elementControl, extraYaws = Parameters.extraYaws, extraPitches = Parameters.extraPitches)

#Default output.
with output:
    clear_output()
    drawState(ax, params)
    plt.show()     


#Defines which function to run when "Show Plot!" button is clicked.
Parameters.showPlotButton.on_click(showPlotClick)
#Defines which function to run when "Set Elements!" button is clicked.
Parameters.setElementButton.on_click(setElementClick)
#Defines which function to run when "Select System" button is clicked.
Parameters.selectSystemButton.on_click(selectSystemClick)

    
#Defining layers of buttons and boxes.
layers = [
    widgets.HBox([Parameters.mainLabel]),
    widgets.HBox([Parameters.systemSelectionDropdown]),
    widgets.HBox([Parameters.selectSystemButton]),
    widgets.HBox([Parameters.elementViewDropdown, Parameters.elementControlDropdown]),
    widgets.HBox([Parameters.setElementButton]),
    widgets.HBox([Parameters.yawTextBox, Parameters.yawUnitsDropdown, Parameters.pitchTextBox, Parameters.pitchUnitsDropdown]),
    widgets.HBox([Parameters.optionsLabel]),
    widgets.HBox([Parameters.plotFontSizeTextBox, Parameters.numberOfLinesTextBox, Parameters.beamColorDropdown, Parameters.showDefaultBeamCheckbox, Parameters.viewXYPlaneCheckbox]),
    widgets.HBox([Parameters.flagFontSizeTextBox, Parameters.numberOfCirclesTextBox,  Parameters.axesTicksTextBox, Parameters.showDisplacementCheckbox]),
    widgets.HBox([Parameters.showPlotButton])
]
#Defining the upper box before the output.
UpperBox = widgets.VBox(layers)
#Defines the whole display box
box = widgets.VBox([UpperBox, output])
#Shows display box
box

In [5]:
"""
TODO for Version 1.0:
    -Test on PRC.
    -Full Example with Flat Mirrors, Lenses, Thin Lenses, Aperture
    -Summary.
    -Save System.
    -Tiny Laser Spot on Large Mirror (Zoom?)\
    -CrossingQ
    -Broken Default
TODO for future versions:
    -Add Analysis tool
    -Add 3D vizualization(?)
    -Multiplot(?)
"""

'\nTODO for Version 1.0:\n    -Test on PRC.\n    -Full Example with Flat Mirrors, Lenses, Thin Lenses, Aperture\n    -Summary.\n    -Save System.\n    -Tiny Laser Spot on Large Mirror (Zoom?)    -CrossingQ\n    -Broken Default\nTODO for future versions:\n    -Add Analysis tool\n    -Add 3D vizualization(?)\n    -Multiplot(?)\n'

In [6]:
b0 = Parameters.beam.copy()
print(b0)
print("\n\n")

b0.interact(Parameters.elements[0])
print(b0)
print("\n\n")
print(b0.collisionQ(Parameters.elements[1]))


Radius Of Curvature : 400
Width : 1
Parameter q : (392.7911630663744+53.212474510349765j)
Wavelength : 0.001064
Index of Refraction : 1
Position : [0 0 0]
Direction : [1. 0. 0.]




Radius Of Curvature : -551.091143502251
Width : 13.605541170688527
Parameter q : (-551.0905832416531+0.5556566742802181j)
Wavelength : 0.001064
Index of Refraction : 1
Position : [4999.89974537    0.            0.        ]
Direction : [-0.98985274  0.142097    0.        ]




True


In [7]:
flags = Parameters.beam.calculateFlags(Parameters.elements)
flags

[]